In [1]:
import csv
import json
import pandas as pd
import sys, getopt, pprint
from pymongo import MongoClient
import zipfile

In [2]:
files =['US_Temp_1995_2020.csv', 'US_Temp_City_List.csv']
file_structures = [
    ["TempReading","Region","Country","State","City","Month","Day","Year","AvgTemperature"],
    ["TempReading","State","City"]
]
segment_names =['US_Temp_1995_2020', 'US_Temp_City_List']
encoding =['UTF-8', 'UTF-8']

for num in range(2):
    filepath = f'Resources/{files[num]}'
    csvfile = open(filepath, 'r', encoding=encoding[num])
    reader = csv.DictReader(csvfile)
    mongo_client=MongoClient()
    db=mongo_client.ClimateData
    if segment_names[num] == 'US_Temp_1995_2020':
        db.US_Temp_1995_2020.drop()
        for each in reader:
            row={}
            for field in file_structures[num]:
                row[field]=each[field]
            db.US_Temp_1995_2020.insert_one(row)
    elif segment_names[num] == 'US_Temp_City_List':
        db.US_Temp_City_List.drop()
        for each in reader:
            row={}
            for field in file_structures[num]:
                row[field]=each[field]
            db.US_Temp_City_List.insert_one(row)
    else:
        print("Error in uploading csvs to database.")

In [3]:
second_file_df_names = ['file1','file2', 'file3', 'file4']
zipfiles = ['US_EPA_1995_2020_first', 'US_EPA_1995_2020_second', 'US_EPA_1995_2020_third', 'US_EPA_1995_2020_fourth']
Second_file_group = ['US_EPA_1995_2020_first','US_EPA_1995_2020_second','US_EPA_1995_2020_third','US_EPA_1995_2020_fourth']
for num in range(4):
    filepath = f'Resources/{zipfiles[num]}.zip'
    file_name = f'{Second_file_group[num]}.csv'
    zf = zipfile.ZipFile(filepath) 
    exec('{} = pd.read_csv(zf.open(file_name))'.format(second_file_df_names[num]))
final_Test = pd.concat([file1, file2, file3, file4], axis=0)
mongo_client=MongoClient()
db=mongo_client.ClimateData
db.US_EPA.drop()
db.US_EPA.insert_many(final_Test.to_dict('records'))